In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorboardX

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [2]:
## Import necessary library.
import matplotlib.pyplot as plt
import torch
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
# from torchvision import tv_tensors
# from torchvision.transforms.v2 import functional as F
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
from torch.utils.data import random_split

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
import pandas as pd
data = pd.read_csv('./train/data.csv')

In [4]:
data['label'].unique().max()

42

In [5]:
from PIL import Image
import numpy as np

np.array(Image.open('../dataset/train/images/'+ '1.jpg').convert('L')).shape

(320, 240)

In [6]:
data

,image-name,label,label-name
0,1.jpg,38,bonus-winter
1,2.jpg,26,characters-8
2,3.jpg,9,dots-9
3,4.jpg,36,bonus-summer
4,5.jpg,28,honors-east
...,...,...,...
624,624.jpg,11,bamboo-2
625,625.jpg,2,dots-2
626,626.jpg,19,characters-1
627,627.jpg,10,bamboo-1


In [7]:
# import os
# for i in range(42):
#     os.makedirs('./label_all/'+str(i))

In [8]:
# import shutil
# for i in range(583, len(data)):
#     source = '../dataset/train/images/'+ data['image-name'][i]
#     target = './label_all/'+ str(data['label'][i])
#     shutil.move(source, target)

In [9]:
# print(i)

In [10]:
# import glob
# import os
# import shutil

# image_list=glob.glob('./label_all/*/*.jpg')
# # print(image_list)
# file_dir='data2'
# if os.path.exists(file_dir):
#     print('true')
#     #os.rmdir(file_dir)
#     shutil.rmtree(file_dir)#删除再建立
#     os.makedirs(file_dir)
# else:
#     os.makedirs(file_dir)

# from sklearn.model_selection import train_test_split

# X_train, X_test = train_test_split(image_list, test_size = 0.20, random_state = 42)

# train_dir='train'
# test_dir='test'

# train_root=os.path.join(file_dir,train_dir)
# test_root=os.path.join(file_dir,test_dir)
# for file in X_train:
#     file_class=file.replace("\\","/").split('/')[-2]
#     file_name=file.replace("\\","/").split('/')[-1]
#     file_class=os.path.join(train_root,file_class)
#     if not os.path.isdir(file_class):
#         os.makedirs(file_class)
#     shutil.copy(file, file_class + '/' + file_name)

    
# for file in X_test:
#     file_class=file.replace("\\","/").split('/')[-2]
#     file_name=file.replace("\\","/").split('/')[-1]
#     file_class=os.path.join(test_root,file_class)
#     if not os.path.isdir(file_class):
#         os.makedirs(file_class)
#     shutil.copy(file, file_class + '/' + file_name)

In [7]:
import torch
from torch.nn.parameter import Parameter
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
import pdb


class Discriminator(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.modu1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Dropout2d(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2),
            nn.LeakyReLU(),
            nn.Dropout2d()
        )
        self.fc1 = nn.Linear(128*6*6, 42) #42 class
        self.fc2 = nn.Linear(128*6*6, 1) #distermin 0,1
        self.bn = nn.BatchNorm2d(128)
    def forward(self, x, if_bn = True, feature = False, cuda = False):
        x = self.modu1(x) # x.shape([1, 128, 6, 6])
        if if_bn:
            x = self.bn(x)
        x = torch.reshape(x, (-1, 128*6*6)) # 4D to 2D
        class_out = self.fc1(x)
        real_fake_out = self.fc2(x)
        # not activiate net
        if feature:
            return class_out, real_fake_out
        else:
            return class_out
        


class Generator(nn.Module):
    def __init__(self,z_dim) -> None:
        super().__init__()
        self.genModel1 = nn.Sequential(
            nn.Linear(100,  256 * 7 * 7 ),
            nn.ReLU(),
            nn.BatchNorm1d(256 * 7 * 7),
        )
        self.genModel2 = nn.Sequential(
            # [128, 28, 28]
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # [64, 14, 14] 
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # [1, 28, 28]
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self,  batch_size, cuda = False):
        x = Variable(torch.rand(batch_size, 100), requires_grad = False, volatile = not self.training)
        x = self.genModel1(x)
        x = torch.reshape(x, (-1, 256, 7, 7))
        x = self.genModel2(x)
        return x
        


In [8]:
import torch
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import pandas as pd
def MahjongLabel(class_num):
    class_tot = [0] * 42
    data = []
    labels = []
    positive_tot = 0
    tot = 0
    raw_dataset =  datasets.ImageFolder('./data2/train', 
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.Grayscale(num_output_channels=1),
                       transforms.ToTensor(),
                   ]))

    
    
    perm = np.random.permutation(raw_dataset.__len__())
    for i in range(raw_dataset.__len__()):
        datum, label = raw_dataset.__getitem__(perm[i])
        # label = label - 1
        try:
            if class_tot[label] < class_num:
                data.append(datum.numpy())
                labels.append(label)
                class_tot[label] += 1
                tot += 1
                # if tot >= 10 * class_num:
                #     break
        except:
            print(label)
    return TensorDataset(torch.FloatTensor(np.array(data)), torch.LongTensor(np.array(labels)))

def MahjongUnlabel():
    raw_dataset =  datasets.ImageFolder('../dataset/unlabel', 
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.Grayscale(num_output_channels=1),
                       transforms.ToTensor(),
                   ]))
    print(raw_dataset)
    return raw_dataset
def MahjongTest():
    return datasets.ImageFolder('./data2/test', 
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.Grayscale(num_output_channels=1),
                       transforms.ToTensor(),
                   ]))



In [9]:
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import math
import pdb
def log_sum_exp(x, axis = 1):
    m = torch.max(x, dim = 1)[0]
    return m + torch.log(torch.sum(torch.exp(x - m.unsqueeze(1)), dim = axis))

In [19]:
# -*- coding:utf-8 -*-
from __future__ import print_function 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader,TensorDataset
import sys
import argparse
import pdb
from torch.utils.tensorboard import *
import os

import random  
import numpy as np
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

# device = "cuda" if torch.cuda.is_available() else "cpu"


# 设置随机种子,保证模型可以复现
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True  # 将cuda加速的随机数生成器设为确定性模式
    torch.backends.cudnn.benchmark = True  # 关闭CuDNN框架的自动寻找最优卷积算法的功能，以避免不同的算法对结果产生影响
    torch.manual_seed(seed)  # pytorch的随机种子
    np.random.seed(seed)  # numpy的随机种子
    random.seed(seed)  # python内置的随机种子


seed_everything(2024)
class ImprovedGAN(object):
    def __init__(self, G, D, labeled, unlabeled, test, args):
        # if os.path.exists(args.savedir):
        #     print('Loading model from ' + args.savedir)
        #     self.G = torch.load(os.path.join(args.savedir, 'G.pkl'))
        #     self.D = torch.load(os.path.join(args.savedir, 'D.pkl'))
        # else:
        #     os.makedirs(args.savedir)
        #     self.G = G
        #     self.D = D
        #     torch.save(self.G, os.path.join(args.savedir, 'G.pkl'))
        #     torch.save(self.D, os.path.join(args.savedir, 'D.pkl'))
        self.G = G
        self.D = D
        self.writer = SummaryWriter(log_dir=args.logdir)
        if args.cuda:
            self.G.cuda()
            self.D.cuda()
        self.labeled = labeled
        self.unlabeled = unlabeled
        self.test = test
        self.Doptim = optim.Adam(self.D.parameters(), lr=args.lr, betas= (args.momentum, 0.999))
        self.Goptim = optim.Adam(self.G.parameters(), lr=args.lr, betas = (args.momentum,0.999))
        self.args = args
    def trainD(self, x_label, y, x_unlabel):
        x_label, x_unlabel, y = Variable(x_label), Variable(x_unlabel), Variable(y, requires_grad = False)
        # print(x_label.shape)
        # print(x_unlabel.shape)
        if self.args.cuda:
            x_label, x_unlabel, y = x_label.cuda(), x_unlabel.cuda(), y.cuda()
        output_label = self.D(x_label, cuda=self.args.cuda)
        output_unlabel = self.D(x_unlabel, cuda=self.args.cuda)
        output_fake = self.D(self.G(x_unlabel.size()[0], cuda = self.args.cuda).view(x_unlabel.size()).detach(), cuda=self.args.cuda)
        prob_label = torch.gather(output_label, 1, y.unsqueeze(1)) # log e^x_label = x_label 
        logz_label, logz_unlabel, logz_fake = log_sum_exp(output_label), log_sum_exp(output_unlabel), log_sum_exp(output_fake)
        loss_supervised = -torch.mean(prob_label) + torch.mean(logz_label)
        loss_unsupervised = 0.5 * (-torch.mean(logz_unlabel) + torch.mean(F.softplus(logz_unlabel))  + # real_data: log Z/(1+Z)
                            torch.mean(F.softplus(logz_fake)) ) # fake_data: log 1/(1+Z)
        loss = loss_supervised + self.args.unlabel_weight * loss_unsupervised
        # print(output_label.max(1)[1])
        # print(y)
        acc = torch.mean((output_label.max(1)[1] == y).float())
        self.Doptim.zero_grad()
        loss.backward()
        self.Doptim.step()
        return loss_supervised.data.cpu().numpy(), loss_unsupervised.data.cpu().numpy(), acc
    
    def trainG(self, x_unlabel):
        fake = self.G(x_unlabel.size()[0], cuda = self.args.cuda).view(x_unlabel.size())
        mom_gen, output_fake = self.D(fake, feature=True, cuda=self.args.cuda)
        mom_unlabel, _ = self.D(Variable(x_unlabel), feature=True, cuda=self.args.cuda)
        mom_gen = torch.mean(mom_gen, dim = 0)
        mom_unlabel = torch.mean(mom_unlabel, dim = 0)
        loss_fm = torch.mean((mom_gen - mom_unlabel) ** 2)
        loss = loss_fm 
        self.Goptim.zero_grad()
        self.Doptim.zero_grad()
        loss.backward()
        self.Goptim.step()
        return loss.data.cpu().numpy()

    def train(self):
        #assert self.unlabeled.__len__() > self.labeled.__len__()
        print(len(self.unlabeled))
        print(len(self.labeled))
        assert type(self.labeled) == TensorDataset
        times = int(np.ceil(self.unlabeled.__len__() * 1. / self.labeled.__len__()))
        t1 = self.labeled.tensors[0].clone()
        t2 = self.labeled.tensors[1].clone()
        print(times)
        print(t1.repeat(times,1,1,1).shape)
        print(t2.repeat(times).shape)
        tile_labeled = TensorDataset(t1.repeat(times,1,1,1),t2.repeat(times))
        gn = 0
        for epoch in range(self.args.epochs):
            self.G.train()
            self.D.train()
            unlabel_loader1 = DataLoader(self.unlabeled, batch_size = self.args.batch_size, shuffle=True, drop_last=True, num_workers = 4)
            unlabel_loader2 = DataLoader(self.unlabeled, batch_size = self.args.batch_size, shuffle=True, drop_last=True, num_workers = 4).__iter__()
            label_loader = DataLoader(tile_labeled, batch_size = self.args.batch_size, shuffle=True, drop_last=True, num_workers = 4).__iter__()
            loss_supervised = loss_unsupervised = loss_gen = accuracy = 0.
            batch_num = 0
            # for d in unlabel_loader1:
            #     print(d)
            #     print('*'*8)
            #     break
            for (unlabel1, _label1) in unlabel_loader1:
                batch_num += 1
                unlabel2, _label2 = next(unlabel_loader2)
                x, y = next(label_loader)
                if args.cuda:
                    x, y, unlabel1, unlabel2 = x.cuda(), y.cuda(), unlabel1.cuda(), unlabel2.cuda()
                ll, lu, acc = self.trainD(x, y, unlabel1)
                loss_supervised += ll
                loss_unsupervised += lu
                accuracy += acc
                lg = self.trainG(unlabel2)
                if epoch > 1 and lg > 1:
                    lg = self.trainG(unlabel2)
                loss_gen += lg
                if (batch_num + 1) % self.args.log_interval == 0:
                    print('Training: %d / %d' % (batch_num + 1, len(unlabel_loader1)))
                    self.writer.add_scalars('loss', {'loss_supervised':ll, 'loss_unsupervised':lu, 'loss_gen':lg}, gn)
                    gn += 1
                    with torch.no_grad():
                        
                        self.writer.add_histogram('real_feature', self.D(Variable(x), cuda=self.args.cuda, feature = True)[0], gn)
                        self.writer.add_histogram('fake_feature', self.D(self.G(self.args.batch_size, cuda = self.args.cuda), cuda=self.args.cuda, feature = True)[0], gn)
                        #self.writer.add_histogram('fc3_bias', self.G.fc3.bias, gn)
                        # self.writer.add_histogram('D_feature_weight', self.D.layers[-1].weight, gn)
                    self.D.train()
                    self.G.train()
            loss_supervised /= batch_num
            loss_unsupervised /= batch_num
            loss_gen /= batch_num
            accuracy /= batch_num
            print("Iteration %d, loss_supervised = %.4f, loss_unsupervised = %.4f, loss_gen = %.4f train acc = %.4f" % (epoch, loss_supervised, loss_unsupervised, loss_gen, accuracy))
            sys.stdout.flush()
            if (epoch + 1) % self.args.eval_interval == 0:
                print("Eval: correct %d / %d"  % (self.eval(), self.test.__len__()))
                print(self.eval()/ self.test.__len__())
                # torch.save(self.G, os.path.join(args.savedir, 'G.pkl'))
                # torch.save(self.D, os.path.join(args.savedir, 'D.pkl'))
                

    def predict(self, x):
        with torch.no_grad():
            ret = torch.max(self.D(Variable(x), cuda=self.args.cuda), 1)[1].data
        return ret

    def eval(self):
        self.G.eval()
        self.D.eval()
        d, l = [], []
        for (datum, label) in self.test:
            d.append(datum)
            l.append(label)
        x, y = torch.stack(d), torch.LongTensor(l)
        if self.args.cuda:
            x, y = x.cuda(), y.cuda()
        pred = self.predict(x)
        return torch.sum(pred == y)
    def draw(self, batch_size):
        self.G.eval()
        return self.G(batch_size, cuda=self.args.cuda)
        # Existing code...

    def save_model(self, save_dir):
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        torch.save(self.G.state_dict(), os.path.join(save_dir, 'G.pth'))
        torch.save(self.D.state_dict(), os.path.join(save_dir, 'D.pth'))
        print("Model saved successfully.")
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='PyTorch Improved GAN')
    parser.add_argument('--batch-size', type=int, default=100, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--epochs', type=int, default=50, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.0005, metavar='LR',
                        help='learning rate (default: 0.001)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--cuda', action='store_true', default=False,
                        help='CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=1, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--eval-interval', type=int, default=1, metavar='N',
                        help='how many epochs to wait before evaling training status')
    parser.add_argument('--unlabel-weight', type=float, default=0.8, metavar='N',
                        help='scale factor between labeled and unlabeled data')
    parser.add_argument('--logdir', type=str, default='./logfile', metavar='LOG_PATH', help='logfile path, tensorboard format')
    parser.add_argument('--savedir', type=str, default='./models', metavar='SAVE_PATH', help = 'saving path, pickle format')
    args = parser.parse_args(args = [])
    args.cuda = args.cuda and torch.cuda.is_available()
    np.random.seed(args.seed)
    gan = ImprovedGAN(Generator(100), Discriminator(), MahjongLabel(42), MahjongUnlabel(), MahjongTest(), args)

    


Dataset ImageFolder
    Number of datapoints: 1050
    Root location: ../dataset/unlabel
    StandardTransform
Transform: Compose(
               Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=True)
               Grayscale(num_output_channels=1)
               ToTensor()
           )


In [20]:
gan.train()

1050
493
3
torch.Size([1479, 1, 28, 28])
torch.Size([1479])
Training: 2 / 10
Training: 3 / 10
Training: 4 / 10
Training: 5 / 10
Training: 6 / 10
Training: 7 / 10
Training: 8 / 10
Training: 9 / 10
Training: 10 / 10
Training: 11 / 10
Iteration 0, loss_supervised = 3.8470, loss_unsupervised = 1.9653, loss_gen = 0.0059 train acc = 0.0390
Eval: correct 6 / 124
tensor(0.0484)
Training: 2 / 10
Training: 3 / 10
Training: 4 / 10
Training: 5 / 10
Training: 6 / 10
Training: 7 / 10
Training: 8 / 10
Training: 9 / 10
Training: 10 / 10
Training: 11 / 10
Iteration 1, loss_supervised = 3.5894, loss_unsupervised = 1.9407, loss_gen = 0.0107 train acc = 0.0920
Eval: correct 24 / 124
tensor(0.1935)
Training: 2 / 10
Training: 3 / 10
Training: 4 / 10
Training: 5 / 10
Training: 6 / 10
Training: 7 / 10
Training: 8 / 10
Training: 9 / 10
Training: 10 / 10
Training: 11 / 10
Iteration 2, loss_supervised = 3.2609, loss_unsupervised = 1.8923, loss_gen = 0.0193 train acc = 0.1560
Eval: correct 29 / 124
tensor(0.2339)

In [20]:
    # Save the model after trainingc
    save_dir = './saved_models'
    gan.save_model(save_dir)

Model saved successfully.


In [ ]:
!tensorboard --logdir=logfile

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.2 at http://localhost:6006/ (Press CTRL+C to quit)
